# Example FRI vs FRII

In [1]:
import h5py

import keras 
from keras.utils import to_categorical
import numpy as np
from pathlib import Path
from sklearn.ensemble import RandomForestClassifier
import tensorflow as tf

from format_data import join_fr_random
from format_data import augment
from format_data import get_fr
from format_data import get_random
from format_data import resize_array
from models import HOGNet
from models import SklearnModel
from query import download_fr_components
from query import download_random

import sklearn

C:\Users\fires\Anaconda3\envs\CSIRO\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
data_path = 'data'
save_path = 'saved_models'

# Setting Seed
seed = 0

# Setting Paths
fr_data_path = Path.cwd() / data_path / 'fr.h5'
random_path = Path.cwd() / data_path / 'random_1000.h5'
everything_path = Path.cwd() / data_path / 'all_temp6.h5'

if not Path(fr_data_path).is_file():
    download_fr_components(fr_data_path)

if not Path(random_path).is_file():
    download_random(random_path, n=1000, seed=seed)

if not Path(everything_path).is_file():
    join_fr_random(fr_data_path, random_path, everything_path)

# Formatting Raw Data

Here, if the files do not already exist, we convert the output of fri-frii-download.ipynb as well as a h5py file of random sources into the required format, and dump them new h5py files.

In [3]:
train_i, test_i = get_fr(everything_path, split_ratio=(1/3), seed=seed)

# Constructing the data generator 
Using the augment_data construxt our keras.preprocessing.image.ImageDataGenerator object to apply randomly augmentations to our data

In [4]:
datagen = augment(rotation_range=180, zoom_range=0.2, shift_range=0.0, flip=True)

# Instantiating Sklearn Classifier
We use the SklearnModel class to construct our random forest classifer. Any other Sklearn Classifier can be used.

# Training Classifer
We train our classifer with the fit method

In [5]:
with h5py.File(everything_path, 'r') as data:
    train_x = np.asarray(data['images'])[train_i]
    test_x = np.asarray(data['images'])[test_i]
    labels = np.asarray(data['labels'])
    images = np.asarray(data['images'])
    
    # Formatting images
    train_x = resize_array(train_x, dim=256)
    train_x = np.expand_dims(train_x, axis=3)
    test_x = resize_array(test_x, dim=256)
    test_x = np.expand_dims(test_x, axis=3)
    
    # Formatting labels
    train_y = labels[train_i] == 2
    test_y = labels[test_i] == 2

In [6]:
def norm(x):
    imgs = np.zeros((x.shape[0], x.shape[1], x.shape[2], x.shape[3]))
    
    for i in range(x.shape[0]):
        img = x[i, :, :, :]
        img -= img.min()
        img /= img.max()
        
        imgs[i, :, :, :] = img
    return imgs

In [7]:
#train_x = norm(train_x)
#test_x = norm(test_x)

In [8]:
#import matplotlib.pyplot as plt

#for i in range(train_x.shape[0]):
    #plt.imshow(train_x[i, :, :, 0]) #cmap="viridis"
    #plt.show()
    #print (train_y[i])

In [9]:
train_x.shape

(914, 256, 256, 1)

In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import shuffle

In [11]:
train_xx, train_yy = shuffle(train_x, train_y, random_state=0)
train_xxx = train_xx.reshape(train_x.shape[0], -1)
test_xx = test_x.reshape(test_x.shape[0], -1)

c = RandomForestClassifier(n_estimators=100)

c = c.fit(train_xxx, train_yy)

In [12]:
c.predict_proba(test_xx)

array([[0.53, 0.47],
       [0.52, 0.48],
       [0.68, 0.32],
       [0.62, 0.38],
       [0.59, 0.41],
       [0.55, 0.45],
       [0.57, 0.43],
       [0.74, 0.26],
       [0.78, 0.22],
       [0.77, 0.23],
       [0.75, 0.25],
       [0.46, 0.54],
       [0.54, 0.46],
       [0.43, 0.57],
       [0.35, 0.65],
       [0.75, 0.25],
       [0.78, 0.22],
       [0.81, 0.19],
       [0.73, 0.27],
       [0.86, 0.14],
       [0.81, 0.19],
       [0.83, 0.17],
       [0.79, 0.21],
       [0.63, 0.37],
       [0.65, 0.35],
       [0.48, 0.52],
       [0.52, 0.48],
       [0.52, 0.48],
       [0.6 , 0.4 ],
       [0.57, 0.43],
       [0.5 , 0.5 ],
       [0.49, 0.51],
       [0.48, 0.52],
       [0.82, 0.18],
       [0.61, 0.39],
       [0.63, 0.37],
       [0.47, 0.53],
       [0.6 , 0.4 ],
       [0.6 , 0.4 ],
       [0.42, 0.58],
       [0.35, 0.65],
       [0.45, 0.55],
       [0.39, 0.61],
       [0.42, 0.58],
       [0.43, 0.57],
       [0.38, 0.62],
       [0.32, 0.68],
       [0.68,

In [13]:
c.score(test_xx, test_y)

0.5589519650655022

In [23]:
from sklearn.naive_bayes import GaussianNB

In [24]:
rft = SklearnModel(GaussianNB, datagen=datagen,
                   nb_augment=3, seed=seed)

In [25]:
rft.fit(train_x, train_y)

In [16]:
#rft.save(path=save_path + 'rft.pk')

# Saving Classifer to Pickle File
Using the save method

In [17]:
#rft.load(path=save_path + "rft.pk")

In [26]:
rft.predict_proba(test_x)

array([[1.00000000e+000, 0.00000000e+000],
       [1.00000000e+000, 0.00000000e+000],
       [1.00000000e+000, 0.00000000e+000],
       [1.00000000e+000, 0.00000000e+000],
       [1.00000000e+000, 0.00000000e+000],
       [1.00000000e+000, 0.00000000e+000],
       [1.00000000e+000, 0.00000000e+000],
       [1.00000000e+000, 0.00000000e+000],
       [1.00000000e+000, 0.00000000e+000],
       [1.00000000e+000, 0.00000000e+000],
       [1.00000000e+000, 0.00000000e+000],
       [1.00000000e+000, 0.00000000e+000],
       [1.00000000e+000, 0.00000000e+000],
       [1.00000000e+000, 0.00000000e+000],
       [1.00000000e+000, 0.00000000e+000],
       [1.00000000e+000, 0.00000000e+000],
       [1.00000000e+000, 0.00000000e+000],
       [1.00000000e+000, 0.00000000e+000],
       [1.00000000e+000, 0.00000000e+000],
       [1.00000000e+000, 0.00000000e+000],
       [1.00000000e+000, 0.00000000e+000],
       [1.00000000e+000, 0.00000000e+000],
       [1.00000000e+000, 0.00000000e+000],
       [1.0

In [27]:
rft.predict(test_x)

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

In [28]:
rft.score(test_x, test_y)

0.5

# Classifying Unseen Samples
First, we load our model with the load method (If the save method has been used on the same script, path is inferred). We test our classifier with the predict method.

# Instantiating HOGNet Model
We use the HOGNet class to construct our custom keras model.

In [32]:
hognet = HOGNet(datagen=datagen, batch_size=50, steps_per_epoch=50, max_epoch=1, patience=5, seed=seed)

C:\Users\fires\Desktop\CSIRO\thursday\thursday\models.py:551: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(8, kernel_size=(1, 1), strides=(1, 1), activation="relu", trainable=True, name="votes", use_bias=False)`
  name="votes")(sin_cos_vec)


# Training Classifer
We train our classifer with the fit method. Model stops training when loss stop decreasing for a set number of epochs (patience argument controls this) or when it reaches the maximum number of epochs. The amount of data augmented by datagen is dictated by the batch_size, and how many batches are generated per epoch is controlled by steps_per_epoch.

In [33]:
hognet.fit(train_x, train_y)

Epoch 1/1
50/50 [==============================] - 133s 3s/step - loss: 0.6922 - acc: 0.5214
Epoch 1/1
50/50 [==============================] - 134s 3s/step - loss: 0.6913 - acc: 0.5248
Epoch 1/1
50/50 [==============================] - 122s 2s/step - loss: 0.6870 - acc: 0.5649


In [34]:
hognet.score(test_x, test_y)

0.5841118816988594

In [35]:
hognet.predict_proba(test_x)

array([[0.4247676 ],
       [0.4231506 ],
       [0.4424319 ],
       [0.44313386],
       [0.44293296],
       [0.44417176],
       [0.44356063],
       [0.46652135],
       [0.4720295 ],
       [0.4613848 ],
       [0.45934078],
       [0.44663775],
       [0.4444744 ],
       [0.44784477],
       [0.44587156],
       [0.43572083],
       [0.43568358],
       [0.43523332],
       [0.43500972],
       [0.43335614],
       [0.43368584],
       [0.43214422],
       [0.4337828 ],
       [0.44176787],
       [0.44188341],
       [0.4796662 ],
       [0.47819304],
       [0.43563756],
       [0.43677413],
       [0.43628323],
       [0.43002245],
       [0.43061176],
       [0.4349066 ],
       [0.44263744],
       [0.4447056 ],
       [0.43689173],
       [0.43105084],
       [0.4310294 ],
       [0.42958334],
       [0.454997  ],
       [0.45393077],
       [0.45380273],
       [0.45233554],
       [0.45402384],
       [0.4535288 ],
       [0.45340073],
       [0.4537165 ],
       [0.470

In [36]:
hognet.predict(test_x)

array([[False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [

# Saving Model Weights to H5py file
Using the save method

In [ ]:
hognet.save(path="data/hognet.h5")

# Classifying Unseen Samples
First, we load our weights into the model with the load method (If the save method has been used on the same script, path is inferred). We test our classifier with the predict method.

In [ ]:
hognet.load()

predictions = hognet.predict_proba(test_x)

#correct = test_y == predictions
#print('Accuracy: {:.02%}'.format(correct.mean()))
print (predictions)